In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [14]:
# define data
train_loader = [3, 4, 5, 6, 7, 8] #data for training
val_loader = [2, 4, 5, 7, 8, 9] #data for validation
test_loader = [1, 2, 3, 4, 5, 6] #true data we want to do calculations on, the newest data

In [4]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()
        self.relu = nn.ReLU()
        
        #Begining of implement here
        
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, hidden_size)
        self.layer3 = nn.Linear(hidden_size, hidden_size)
        self.layer4 = nn.Linear(hidden_size, hidden_size)
        self.layer5 = nn.Linear(hidden_size, output_size)

    def forward(self, x):   
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        out = self.relu(out)
        out = self.layer3(out)
        out = self.relu(out)
        out = self.layer4(out)
        out = self.relu(out)
        out = self.layer5(out)
        return out

In [11]:
#Train model
def train(model, criterion, optimizer, train_loader, val_loader, n_epochs):
    train_acc, val_acc = [], []
    for epoch in range(n_epochs):
        model.train()
        correct, total = 1, 1
        for idx, (inputs, labels) in enumerate(train_loader): #This must be changed
            #128, 1, 28, 28 to 
            #128, 784
            inputs, labels = inputs.reshape(128, 28*28), labels #reshaping the input vector and sending to device

            #Forward pass
            output = model(inputs)
            loss = criterion(output, labels)
            
            #Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            #Predicted data
            output = model(inputs)
            _, predicted = torch.max(output, 1)

            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            if idx % 100 == 0:
                print(f'epoch: {epoch} / {n_epochs}, step {idx}, loss: {loss:.2f}, train_acc: {(correct/total):.2f}')
    
        train_acc.append(correct/total)

        correct, total = 1, 1
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.reshape(-1, 28*28), labels
                
                #Obtaining outpuuts and preiction for evalutaion of model
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)

                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        val_acc.append(correct/total)
        print(f'Accuracy = {train_acc}')

    return train_acc, val_acc

In [12]:
#Test function
def test(model, test_loader):
    model.eval()
    correct, total = 1, 1
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.reshape(-1, 28*28), labels #reshaping the input vector and sending input and label to device

            #Obtaining output and prediction 
            output = model(inputs)
            _, predicted = torch.max(output, 1)

            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
    return correct/total

In [9]:
#Hyperparameters
input_size = 5 #size of vector containing parameters of option price
hidden_size = 32
output_size = 1 #output is a price of a option at a specific date
n_epochs = 5 #number of runs through the neural network
lr = 0.01

In [15]:
model = MLP(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
train_acc, val_acc = train(model, criterion, optimizer, train_loader, val_loader, n_epochs)
test_acc = test(model, test_loader)
plt.plot(train_acc, label='Train Acc')
plt.plot(val_acc, label='Val acc')
plt.legend(loc='lower right')
plt.title(f'Test acc: {test_acc}')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

TypeError: cannot unpack non-iterable int object